# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
total_sum = 0
for sqrt_value in generator:
    print(sqrt_value)
    total_sum = sqrt_value + total_sum

print(f'Total sum - {total_sum}')


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Total sum - 8.382332347441762


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
total_sum = 0
for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)





{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [5]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_one_data",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people_two_data",
										write_disposition="replace")

print(info)


Pipeline dlt_ipykernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\swath\Downloads\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707356890.698148 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\swath\Downloads\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707356891.6223266 is LOADED and contains no failed jobs


In [6]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))



Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_one_data     │
│ people_two_data     │
└─────────────────────┘

In [9]:
print("\n\n\n Sum of ages of people 1 data")

total_age = conn.sql("""
SELECT SUM(AGE)
FROM people_one_data 
""").df()
display(total_age)




 Sum of ages of people 1 data


,sum(AGE)
0,140.0


In [7]:
# and the data

print("\n\n\n people_one_data table below:")

rides = conn.sql("SELECT * FROM people_one_data").df()
display(rides)

print("\n\n\n people_two_data table below:")

passengers = conn.sql("SELECT * FROM people_two_data").df()
display(passengers)

# As you can see, the same data was loaded in both cases.




 people_one_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707356890.698148,4WYi5nGOBSBlBQ
1,2,Person_2,27,City_A,1707356890.698148,wH31PzBiqMJQAg
2,3,Person_3,28,City_A,1707356890.698148,AlynjY1dwV25xw
3,4,Person_4,29,City_A,1707356890.698148,f7pePmrxheP77g
4,5,Person_5,30,City_A,1707356890.698148,G0rzKL2R/v0DnQ





 people_two_data table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,3,Person_3,33,City_B,Job_3,1707356891.6223266,QTVmU4PdOCrW4g
1,4,Person_4,34,City_B,Job_4,1707356891.6223266,Pofy9Nh37gnJTA
2,5,Person_5,35,City_B,Job_5,1707356891.6223266,0bBtFMeYEx3sDg
3,6,Person_6,36,City_B,Job_6,1707356891.6223266,WHDHQPvZXJHRBQ
4,7,Person_7,37,City_B,Job_7,1707356891.6223266,YYQqwbLf2iJjkQ
5,8,Person_8,38,City_B,Job_8,1707356891.6223266,357/2MXcRd4Skw


In [10]:
info = generators_pipeline.run(people_2(),
										table_name="people_one_data",
										write_disposition="append")

print(info)


Pipeline dlt_ipykernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\swath\Downloads\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707357486.818756 is LOADED and contains no failed jobs


In [11]:
data_people = conn.sql("SELECT * FROM people_one_data").df()
display(data_people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707356890.698148,4WYi5nGOBSBlBQ,None
1,2,Person_2,27,City_A,1707356890.698148,wH31PzBiqMJQAg,None
2,3,Person_3,28,City_A,1707356890.698148,AlynjY1dwV25xw,None
3,4,Person_4,29,City_A,1707356890.698148,f7pePmrxheP77g,None
4,5,Person_5,30,City_A,1707356890.698148,G0rzKL2R/v0DnQ,None
5,3,Person_3,33,City_B,1707357486.818756,PBY7H2Alg8WlNA,Job_3
6,4,Person_4,34,City_B,1707357486.818756,VQ/j4RhBRBK6hg,Job_4
7,5,Person_5,35,City_B,1707357486.818756,Eya3eJfaz+dfwg,Job_5
8,6,Person_6,36,City_B,1707357486.818756,Q9YKEnc7ewv6gw,Job_6
9,7,Person_7,37,City_B,1707357486.818756,7LHKkedhkym9Hw,Job_7


In [12]:
total_age_sum = conn.sql("SELECT SUM(AGE) FROM people_one_data").df()
display(total_age_sum)

,sum(AGE)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [17]:
pip install dlt[duckdb]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
# to do: homework :)

#generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_one_data_MERGE",
										write_disposition="replace")




In [27]:
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\swath\Downloads\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707359036.9146364 is LOADED and contains no failed jobs


In [28]:

print('Loaded tables: ')
display(conn.sql("show tables"))


Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ people_one_data       │
│ people_one_data_merge │
│ people_two_data       │
└───────────────────────┘

In [29]:
print('Loaded tables: ')
display(conn.sql("show tables"))

check = conn.sql("SELECT * FROM people_one_data_MERGE").df()
display(check)

Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ people_one_data       │
│ people_one_data_merge │
│ people_two_data       │
└───────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707359036.9146364,xtzr/7WHI04KXA,None
1,2,Person_2,27,City_A,1707359036.9146364,9mJJBdRjTRqizA,None
2,3,Person_3,28,City_A,1707359036.9146364,vLEebgNOLdt7xA,None
3,4,Person_4,29,City_A,1707359036.9146364,Dou4SV5ezcU29g,None
4,5,Person_5,30,City_A,1707359036.9146364,D2etAL/AOoCIaA,None


In [30]:
info = generators_pipeline.run(people_2(),
										table_name="people_one_data_MERGE",
										write_disposition="merge", primary_key="id")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\swath\Downloads\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707359239.5119662 is LOADED and contains no failed jobs


In [31]:

print('Loaded tables: ')
display(conn.sql("show tables"))

check = conn.sql("SELECT * FROM people_one_data_MERGE").df()
display(check)

Loaded tables: 


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ _dlt_loads            │
│ _dlt_pipeline_state   │
│ _dlt_version          │
│ people_one_data       │
│ people_one_data_merge │
│ people_two_data       │
└───────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707359036.9146364,xtzr/7WHI04KXA,None
1,2,Person_2,27,City_A,1707359036.9146364,9mJJBdRjTRqizA,None
2,4,Person_4,34,City_B,1707359239.5119662,FNvQymTNSi5KeQ,Job_4
3,5,Person_5,35,City_B,1707359239.5119662,4ub8sHBLsoy5EA,Job_5
4,8,Person_8,38,City_B,1707359239.5119662,ezXjsIPA8NmsFg,Job_8
5,3,Person_3,33,City_B,1707359239.5119662,FOR9GfJWERZoRw,Job_3
6,6,Person_6,36,City_B,1707359239.5119662,1W66tMYbjx6NHQ,Job_6
7,7,Person_7,37,City_B,1707359239.5119662,/ea/oB/ucJ0PfQ,Job_7


In [32]:
total_age_sum = conn.sql("SELECT SUM(AGE) FROM people_one_data_MERGE").df()
display(total_age_sum)

,sum(AGE)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX